<a href="https://colab.research.google.com/github/clapg/meu-portfolio-clarice/blob/main/EDA_consolidado_Squad8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA - Squad 08

### Leitura das bases

In [ ]:
# Criar dicionário para armazenar DataFrames
dataframes = {}
caminho_root = '/Volumes/hackathon_2025/default/source/'

# base_dados_cadastrais
try:
    df = spark.read.parquet(f'{caminho_root}base_dados_cadastrais/')
    dataframes['base_dados_cadastrais'] = df
    print(f'base_dados_cadastrais:{df.count():,} linhas, {len(df.columns)} colunas')
except Exception as e:
    print(f'Erro: {str(e)[:100]}')

# base_score_bureau_movel
try:
    df = spark.read.parquet(f'{caminho_root}base_score_bureau_movel/')
    dataframes['base_score_bureau_movel'] = df
    print(f'base_score_bureau_movel:{df.count():,} linhas, {len(df.columns)} colunas')
except Exception as e:
    print(f'Erro: {str(e)[:100]}')

# base_score_bureau_movel_full (NOVA)
try:
    df = spark.read.parquet(f'{caminho_root}base_score_bureau_movel_full/')
    dataframes['base_score_bureau_movel_full'] = df
    print(f'base_score_bureau_movel_full:{df.count():,} linhas, {len(df.columns)} colunas')
except Exception as e:
    print(f'Erro: {str(e)[:100]}')

# base_telco
try:
    df = spark.read.parquet(f'{caminho_root}base_telco/')
    dataframes['base_telco'] = df
    print(f'base_telco:{df.count():,} linhas, {len(df.columns)} colunas')
except Exception as e:
    print(f'Erro: {str(e)[:100]}')

# csv de bases_recarga
csv_files = [
    'BI_DIM_CANAL_AQUISICAO_CREDITO',
    'BI_DIM_FORMA_PAGAMENTO',
    'BI_DIM_INSTITUICAO',
    'BI_DIM_PLANO_PRECO',
    'BI_DIM_PLATAFORMA',
    'BI_DIM_PROMOCAO_CREDITO',
    'BI_DIM_STATUS_PLATAFORMA',
    'BI_DIM_TECNOLOGIA',
    'BI_DIM_TIPO_CREDITO',
    'BI_DIM_TIPO_INSERCAO',
    'BI_DIM_TIPO_RECARGA'
]

for csv_file in csv_files:
    try:
        df = spark.read.csv(f'{caminho_root}bases_recarga/{csv_file}.csv', header=True, inferSchema=True)
        dataframes[csv_file] = df
        print(f'{csv_file}: {df.count():,} linhas')
    except Exception as e:
        print(f'{csv_file}: {str(e)[:50]}')

# BI_DIM_TIPO_FATURAMENTO
try:
    df = spark.read.csv(f'{caminho_root}book_atraso/BI_DIM_TIPO_FATURAMENTO.csv', header=True, inferSchema=True)
    dataframes['BI_DIM_TIPO_FATURAMENTO'] = df
    print(f'BI_DIM_TIPO_FATURAMENTO: {df.count():,} linhas')
except Exception as e:
    print(f'Erro: {str(e)[:100]}')

# parquet - book_atraso/dados_faturamento/
try:
    df = spark.read.parquet(f'{caminho_root}book_atraso/dados_faturamento/')
    dataframes['dados_faturamento'] = df
    print(f'dados_faturamento: {df.count():,} linhas, {len(df.columns)} colunas')
except Exception as e:
    print(f'Erro: {str(e)[:100]}')

# parquet - bases_recarga/BI_FP_ASS_RECARGA_CMV_NOVA/
try:
    df = spark.read.parquet(f'{caminho_root}bases_recarga/BI_FP_ASS_RECARGA_CMV_NOVA/')
    dataframes['BI_FP_ASS_RECARGA_CMV_NOVA'] = df
    print(f'BI_FP_ASS_RECARGA_CMV_NOVA: {df.count():,} linhas, {len(df.columns)} colunas')
except Exception as e:
    print(f'Erro: {str(e)[:100]}')

# parquet - book_pagamento/dados_pagamento/
import os
caminho_pagamento = f'{caminho_root}book_pagamento/dados_pagamento/'

try:
    # Tentar carregar com wildcard
    df = spark.read.parquet(f'{caminho_pagamento}*.parquet')
    dataframes['dados_pagamento'] = df
    print(f'dados_pagamento (wildcard): {df.count():,} linhas, {len(df.columns)} colunas')
except Exception as e:
    print(f'Wildcard falhou, tentando listar arquivos...')

    # Se wildcard falhar, tentar carregar cada arquivo
    try:
        arquivos = os.listdir(caminho_pagamento)
        print(f'Encontrados {len(arquivos)} arquivos:')

        for arquivo in arquivos:
            if arquivo.endswith('.parquet'):
                print(f"     → {arquivo}")
                try:
                    caminho_completo = os.path.join(caminho_pagamento, arquivo)
                    df = spark.read.parquet(caminho_completo)

                    # Usar nome do arquivo como chave
                    nome_tabela = arquivo.replace('.parquet', '').replace('-c000', '')
                    if nome_tabela not in dataframes:  # Evitar duplicatas
                        dataframes[nome_tabela] = df

                    print(f'{nome_tabela}: {df.count():,} linhas')
                except Exception as e2:
                    print(f'Erro: {str(e2)[:50]}')
    except Exception as e:
        print(f'Erro ao listar diretório: {str(e)[:100]}')

# Converter todos os nomes dos dataframes para minúsculas
dataframes = {chave.lower(): df for chave, df in dataframes.items()}

### Metadados

In [ ]:
# importar biblioteca
from pyspark.sql.functions import col, count, when, lit


# Lista para armazenar metadados
metadata = []

# Iterar sobre cada dataframe
for nome_df, df in dataframes.items():
    total_linhas = df.count()

    # Analisar cada coluna
    for coluna in df.columns:
        tipo = df.schema[coluna].dataType

        # Contar nulos
        qt_nulos = df.filter(col(coluna).isNull()).count()
        percent_nulos = (qt_nulos / total_linhas) * 100 if total_linhas > 0 else 0

        # Cardinalidade (valores únicos)
        cardinalidade = df.select(coluna).distinct().count()

        # Adicionar à lista
        metadata.append({
            'nome_dataframe': nome_df,
            'nome_variavel': coluna,
            'tipo': str(tipo),
            'qt_nulos': int(qt_nulos),
            'percent_nulos': round(percent_nulos, 2),
            'cardinalidade': int(cardinalidade)
        })

# Converter para Spark DataFrame
df_resultado = spark.createDataFrame(metadata)

# Exibir
display(df_resultado)